<a href="https://colab.research.google.com/github/dansojo/Super-Resolution/blob/main/FSRCNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FSRCNN (Fast Super-Resolution Convolutional Neural Network)

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import matplotlib.image as img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU
from tensorflow.keras.optimizers import Adam

In [ ]:
def load_images(lr_directory, hr_directory, target_size=(512, 512)):
    x_train = []
    y_train = []

    # 파일 목록을 안전하게 가져오기
    try:
        lr_filenames = sorted(os.listdir(lr_directory))
        hr_filenames = sorted(os.listdir(hr_directory))
    except FileNotFoundError:
        print("Directory not found")
        return [], []

    for lr_file, hr_file in zip(lr_filenames, hr_filenames):
        try:
            lr_image_path = os.path.join(lr_directory, lr_file)
            hr_image_path = os.path.join(hr_directory, hr_file)

            lr_img = Image.open(lr_image_path).resize(target_size, Image.BICUBIC)
            hr_img = Image.open(hr_image_path).resize((2048, 2048), Image.BICUBIC)

            x_train.append(np.array(lr_img))
            y_train.append(np.array(hr_img))
        except (IOError, OSError):
            print(f"Error processing {lr_file} or {hr_file}")

    if len(x_train) > 0 and len(y_train) > 0:
        x_train = np.array(x_train) / 255.0
        y_train = np.array(y_train) / 255.0

    return x_train, y_train

In [ ]:
def FSRCNN(scale_factor, num_channels=1, d=56, s=12, m=4):
    # Input layer
    input_layer = Input(shape=(None, None, num_channels), name='input_layer')

    # Feature extraction
    x = Conv2D(d, (5, 5), padding='same', activation='relu', name='feature_extraction')(input_layer)

    # Shrinking
    x = Conv2D(s, (1, 1), padding='same', activation='relu', name='shrinking')(x)

    # Non-linear mapping
    for i in range(m):
        x = Conv2D(s, (3, 3), padding='same', activation='relu', name=f'non_linear_mapping_{i+1}')(x)

    # Expanding
    x = Conv2D(d, (1, 1), padding='same', activation='relu', name='expanding')(x)

    # Deconvolution
    x = Conv2D(num_channels, (9, 9), strides=(scale_factor, scale_factor), padding='same', activation='linear', name='reconstruction')(x)

    # Create model
    model = Model(inputs=input_layer, outputs=x)
    return model

In [ ]:
# 이미지 데이터셋 로드 경로
dataset_hr_directory = '/content/drive/MyDrive/Super-Resolution/SR_data_set/train/hr'
dataset_lr_directory = '/content/drive/MyDrive/Super-Resolution/SR_data_set/train/lr'
x_train, y_train = load_images(dataset_lr_directory, dataset_hr_directory)

In [ ]:
# Create the FSRCNN model
scale_factor = 4  # Example scale factor
fsrcnn_model = FSRCNN(scale_factor=scale_factor)

# Compile the model
fsrcnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
fsrcnn_model.fit(x_train, y_train, batch_size=16, epochs=100)

# After training, you might want to save the model or visualize the results
fsrcnn_model.save('fsrcnn_model_v1.h5')
